# Quickstart

:::{note}
For installation instructions, please refer to the [installation guide](./installation).
:::


An agent is a software entity that communicates via messages, maintains its own state, and performs actions in response to received messages or changes in its state. 

In AgentChat, agents can be rapidly implemented using preset agent configurations. To illustrate this, we will begin with creating an agent that can address tasks by responding to messages it receives. 

In [1]:
import logging
from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.agents import CodingAssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat, MaxMessageTermination
from autogen_core.components.models import OpenAIChatCompletionClient

logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)

# define an agent
writing_agent = CodingAssistantAgent(
    name="writing_agent",
    system_message="You are a helpful assistant that can respond to questions",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-2024-08-06"),
)

# add the agent to a team
agent_team = RoundRobinGroupChat([writing_agent])
result = await agent_team.run(
    task="Write a 5 line haiku on international space station",
    termination_condition=MaxMessageTermination(max_messages=1),
)
print("\n", result)


--------------------------------------------------------------------------- 
[2024-10-19T20:51:24.752274]:

Write a 5 line haiku on international space station
--------------------------------------------------------------------------- 
[2024-10-19T20:51:25.926555], writing_agent:

In orbit we glide,  
A dance of earthbound nations,  
Stars our common home.  

Floating dreams connect,  
Horizons without borders.  
--------------------------------------------------------------------------- 
[2024-10-19T20:51:25.926879], Termination:

Maximal number of messages 1 reached, current message count: 1
 TeamRunResult(messages=[TextMessage(source='user', content='Write a 5 line haiku on international space station'), TextMessage(source='writing_agent', content='In orbit we glide,  \nA dance of earthbound nations,  \nStars our common home.  \n\nFloating dreams connect,  \nHorizons without borders.  ')])


The code snippet above introduces two high level concepts in AgentChat: `Agent` and `Team`. An Agent helps us define what actions are taken when a message is received, and a Team helps us define the rules for how agents interact with each other (e.g., the order in which agents are called, when the conversation ends).

## What's Next?

Now that you have a basic understanding of how to define an agent and a team, consider following the [tutorial](./tutorial/index) for a walkthrough on other features of AgentChat.

